In [23]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    image.flags.writeable = False                  
    results = model.process(image)                
    image.flags.writeable = True                 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [4]:
def extract_keypoints(results):
    # Initialize empty arrays
    lh = np.zeros(21 * 3)
    rh = np.zeros(21 * 3)

    if results.multi_hand_landmarks and results.multi_handedness:
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            label = results.multi_handedness[idx].classification[0].label  # 'Left' or 'Right'
            hand_kp = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]).flatten()
            
            if label == 'Left':
                lh = hand_kp
            elif label == 'Right':
                rh = hand_kp

    return np.concatenate([lh, rh])


In [ ]:
DATA_PATH = os.path.join('ASL_Data')
actions = np.array(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z', 'box', 'chocolate', 'clean', 'cup', 'divorce', 'dog','fear','floor','friday','how','like','marriage','name','pencil','single','their','what','where','yeah','you','your'])
n_sequences = 20
s_length = 30


In [ ]:
for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    os.makedirs(action_path, exist_ok=True)

    existing_dirs = os.listdir(action_path)
    if existing_dirs:
        dirmax = np.max(np.array(existing_dirs).astype(int))
    else:
        dirmax = -1  # if no existing folders, start at 0
    for sequence in range(n_sequences):  # 0 to 19
        try: 
            os.makedirs(os.path.join(action_path, str(dirmax + 1 + sequence)))
        except:
            pass

In [42]:
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

In [ ]:


cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands:

    for action in actions:
        for sequence in range(n_sequences):
            for frame_num in range(s_length):

                ret, frame = cap.read()

                if not ret:
                    break
                
                image, res = mediapipe_detection(frame, hands)
                if res.multi_hand_landmarks:
                    for hand_landmarks in res.multi_hand_landmarks:
                        mp_draw.draw_landmarks(
                            image,
                            hand_landmarks,
                            mp_hands.HAND_CONNECTIONS)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Feed', image)
                    cv2.waitKey(800)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('Feed', image)
                
                keypoints = extract_keypoints(res)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(15) & 0xFF == ord('z'):
                    break
        cv2.waitKey(500)
    cap.release()
cv2.destroyAllWindows()

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []

In [10]:

for action in actions:
    for sequence in range(n_sequences):
        window = []

        for frame_num in range(s_length):
            filename = f"{frame_num}.npy"
            filepath = os.path.join(DATA_PATH, action, str(sequence), filename)
            res = np.load(filepath)
            window.append(res)

        sequences.append(window)
        labels.append(label_map[action])

In [17]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [31]:
model = Sequential()
model.add(LSTM(8, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(16, return_sequences=True, activation='relu'))
model.add(LSTM(8, return_sequences=False, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [33]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
28/28 [==============================] - 3s 13ms/step - loss: 3.8526 - categorical_accuracy: 0.0157
Epoch 2/2000
28/28 [==============================] - 0s 14ms/step - loss: 3.8384 - categorical_accuracy: 0.0078
Epoch 3/2000
28/28 [==============================] - 0s 14ms/step - loss: 3.7966 - categorical_accuracy: 0.0168
Epoch 4/2000
28/28 [==============================] - 0s 14ms/step - loss: 3.7161 - categorical_accuracy: 0.0325
Epoch 5/2000
28/28 [==============================] - 0s 13ms/step - loss: 3.6077 - categorical_accuracy: 0.0549
Epoch 6/2000
28/28 [==============================] - 0s 13ms/step - loss: 3.5108 - categorical_accuracy: 0.0470
Epoch 7/2000
28/28 [==============================] - 0s 13ms/step - loss: 3.4430 - categorical_accuracy: 0.0482
Epoch 8/2000
28/28 [==============================] - 0s 13ms/step - loss: 3.3667 - categorical_accuracy: 0.0493
Epoch 9/2000
28/28 [==============================] - 0s 13ms/step - loss: 3.2792 - categorical_

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 8)             4320      
                                                                 
 lstm_7 (LSTM)               (None, 30, 16)            1600      
                                                                 
 lstm_8 (LSTM)               (None, 8)                 800       
                                                                 
 dense_6 (Dense)             (None, 8)                 72        
                                                                 
 dense_7 (Dense)             (None, 47)                423       
                                                                 
Total params: 7215 (28.18 KB)
Trainable params: 7215 (28.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [35]:
res = model.predict(X_test)

2/2 [==============================] - 0s 6ms/step


In [36]:
actions[np.argmax(res[4])]

'chocolate'

In [37]:
actions[np.argmax(y_test[4])]

'chocolate'

In [38]:
from scipy import stats

In [39]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [48]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 

with mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        print(results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5: 
                            sentence = sentence[-5:]

                        # Viz probabilities
            image = prob_viz(res, actions, image, colors)
                        
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
